## Zachary Anderson - Artificial Intelligence 001C 1212
# Intelligent Agents: Reflex Agents for the Vacuum-cleaner World


## Instructions

Total Points: undergrad 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with 

* your implementation,
* documentation including a short discussion of how your implementation works and your design choices, and
* experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. 

Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square is cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can `clean` the current square or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [1]:
from numpy import random

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return random.choice(actions)

In [2]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'south'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

This simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns the performance measure which is here the number of cleaned squares (since all squares are constantly dirty, it is the number of `suck` actions by the agent). 

In [3]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with 20 steps.

In [4]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: suck
step 1 - action: south
step 2 - action: north
step 3 - action: suck
step 4 - action: west
step 5 - action: east
step 6 - action: north
step 7 - action: north
step 8 - action: suck
step 9 - action: east
step 10 - action: south
step 11 - action: south
step 12 - action: suck
step 13 - action: west
step 14 - action: east
step 15 - action: west
step 16 - action: suck
step 17 - action: south
step 18 - action: east
step 19 - action: suck


6

# Tasks

_Submission Instructions:_ Use this notebook to prepare your submission. Complete this section with your code and results. You can add additional Markdown blocks for your description, comments in the code and use mathplotlib to produce charts. 

_Note:_ Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design. 


## Task 1: Implement a simulation environment [2 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty.
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment needs to work with the simple randomized agent program from above and then it can be used for your agent implementation in the tasks below.

In [5]:
# Simulation environment
from numpy import array

#function that updates the bumpers, when agent is next to a wall
def update_bumpers(robot_pos, dims):
    bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}
    if(robot_pos[0] == 0): bumpers.update({"north": True})
    if(robot_pos[0] == dims-1): bumpers.update({"south": True})
    if(robot_pos[1] == 0): bumpers.update({"west": True})
    if(robot_pos[1] == dims-1): bumpers.update({"east": True})
    return bumpers

#function that updates the position. The robot does not move the bumper is activated.
def update_position(robot_pos, action, bumpers):
    if(action == "north"):
        if(bumpers["north"] == False): robot_pos[0] = robot_pos[0]-1
    if(action == "south"):
        if(bumpers["south"] == False): robot_pos[0] = robot_pos[0]+1
    if(action == "west"):
        if(bumpers["west"] == False): robot_pos[1] = robot_pos[1]-1
    if(action == "east"):
        if(bumpers["east"] == False): robot_pos[1] = robot_pos[1]+1
    return robot_pos

#environment function that handles our agent
def square_environment(agent, dims, max_steps, verbose = True):
    num_cleaned = 0
    num_dirty = 0
    energy_used = 0
    
    #generating room with each tile having 20% chance of being dirty
    room = array([[random.choice([False, True], p=[0.8, 0.2]) for x in range(dims)] for y in range(dims)])
    if(verbose): print("Shape of room: ", room.shape)
    if(verbose): print("Generated room:")
        
    #count number of dirty tiles
    for x in room: 
        for y in x:
            if(y == True): num_dirty = num_dirty + 1
        if(verbose): 
            print(x)
    if (verbose): print("room has", num_dirty , "dirty tiles")
        
    #randomly assign robot position in environment
    robot_pos = array([random.randint(dims-1), random.randint(dims-1)])
    if(verbose): print("Robot starts at tile: (", robot_pos[0], ",", robot_pos[1], ")")
        
    #initialize robot bumpers
    bumpers = update_bumpers(robot_pos, dims)
    
    #Robot does actions until max_steps or all tiles clean
    for i in range(max_steps):
        if(num_dirty == 0): break
        dirty = room[robot_pos[0],robot_pos[1]]
        action = agent(bumpers, dirty)
        energy_used = energy_used + 1
        if (verbose): print("step", i , "- action:", action, "position: (", robot_pos[0], ",", robot_pos[1], ")")
        # If robot sucks and tile is dirty, update counters
        if (action == "suck"): 
            if(room[robot_pos[0],robot_pos[1]]):
                room[robot_pos[0],robot_pos[1]] = False
                num_cleaned = num_cleaned + 1
                num_dirty = num_dirty - 1
        # If robot is unable to do anything, break
        elif (action == "noop"):
            print("Robot is unable to do anything. Stopping robot.")
            break
        # Update the position and bumpers for next action
        else:
            robot_pos = update_position(robot_pos, action, bumpers)
            bumpers = update_bumpers(robot_pos, dims)
            
        
    print("Number of tiles cleaned: ", num_cleaned, ".")
    print("Number of dirty tiles remaining = ", num_dirty, ".")
    print("Amount of energy units consumed: ", energy_used, ".")
    return num_cleaned

square_environment(simple_randomized_agent, dims=5, max_steps = 1000)

Shape of room:  (5, 5)
Generated room:
[False False False False False]
[ True False False False False]
[False False False False False]
[False  True  True False  True]
[ True False False  True False]
room has 6 dirty tiles
Robot starts at tile: ( 2 , 1 )
step 0 - action: south position: ( 2 , 1 )
step 1 - action: west position: ( 3 , 1 )
step 2 - action: suck position: ( 3 , 0 )
step 3 - action: south position: ( 3 , 0 )
step 4 - action: south position: ( 4 , 0 )
step 5 - action: west position: ( 4 , 0 )
step 6 - action: west position: ( 4 , 0 )
step 7 - action: suck position: ( 4 , 0 )
step 8 - action: south position: ( 4 , 0 )
step 9 - action: east position: ( 4 , 0 )
step 10 - action: suck position: ( 4 , 1 )
step 11 - action: south position: ( 4 , 1 )
step 12 - action: north position: ( 4 , 1 )
step 13 - action: west position: ( 3 , 1 )
step 14 - action: north position: ( 3 , 0 )
step 15 - action: north position: ( 2 , 0 )
step 16 - action: suck position: ( 1 , 0 )
step 17 - action:

step 204 - action: west position: ( 0 , 4 )
step 205 - action: suck position: ( 0 , 3 )
step 206 - action: west position: ( 0 , 3 )
step 207 - action: north position: ( 0 , 2 )
step 208 - action: suck position: ( 0 , 2 )
step 209 - action: south position: ( 0 , 2 )
step 210 - action: suck position: ( 1 , 2 )
step 211 - action: east position: ( 1 , 2 )
step 212 - action: suck position: ( 1 , 3 )
step 213 - action: suck position: ( 1 , 3 )
step 214 - action: suck position: ( 1 , 3 )
step 215 - action: east position: ( 1 , 3 )
step 216 - action: suck position: ( 1 , 4 )
step 217 - action: suck position: ( 1 , 4 )
step 218 - action: north position: ( 1 , 4 )
step 219 - action: suck position: ( 0 , 4 )
step 220 - action: west position: ( 0 , 4 )
step 221 - action: suck position: ( 0 , 3 )
step 222 - action: west position: ( 0 , 3 )
step 223 - action: west position: ( 0 , 2 )
step 224 - action: west position: ( 0 , 1 )
step 225 - action: west position: ( 0 , 0 )
step 226 - action: east posit

6

## Task 2:  Implement a simple reflex agent [1 Point] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ The agent cannot directly use variable in the environment. It only gets the percepts as the arguments to the agent program function.

In [6]:
# Simple reflex agent that sucks if dirty
# Agent moves in a non-blocked direction if not dirty
# Agent returns "noop" if all bumpers activated
def simple_reflex_agent(bumpers, dirty):
    if(dirty): return "suck"
    reflex_actions = ["north", "east", "west", "south"]
    if(bumpers["north"]): reflex_actions.remove("north")
    if(bumpers["east"]): reflex_actions.remove("east")
    if(bumpers["west"]): reflex_actions.remove("west")
    if(bumpers["south"]): reflex_actions.remove("south")
    #If for some reason all bumpers are set to True, robot should do nothing
    if(len(reflex_actions) == 0): return "noop"
    return random.choice(reflex_actions)

square_environment(simple_reflex_agent, dims=5, max_steps = 1000)

Shape of room:  (5, 5)
Generated room:
[False False False False False]
[False False False False False]
[False False False  True False]
[ True  True  True False False]
[False False False False False]
room has 4 dirty tiles
Robot starts at tile: ( 3 , 3 )
step 0 - action: south position: ( 3 , 3 )
step 1 - action: west position: ( 4 , 3 )
step 2 - action: north position: ( 4 , 2 )
step 3 - action: suck position: ( 3 , 2 )
step 4 - action: north position: ( 3 , 2 )
step 5 - action: west position: ( 2 , 2 )
step 6 - action: east position: ( 2 , 1 )
step 7 - action: north position: ( 2 , 2 )
step 8 - action: east position: ( 1 , 2 )
step 9 - action: north position: ( 1 , 3 )
step 10 - action: east position: ( 0 , 3 )
step 11 - action: west position: ( 0 , 4 )
step 12 - action: west position: ( 0 , 3 )
step 13 - action: south position: ( 0 , 2 )
step 14 - action: west position: ( 1 , 2 )
step 15 - action: north position: ( 1 , 1 )
step 16 - action: south position: ( 0 , 1 )
step 17 - action:

4

## Task 3: Implement a model-based reflex agent [3 Point]

This agent keeps track of the location and remembers where it has cleaned. Assume the agent knows how many squares the room has. It can move to a corner to determine its location and then is able to use more advanced navigation.

Describe how you define the __agent state__ and how your agent works before implementing it. _Note on implementing the state in Python:_ [Examples](https://github.com/mhahsler/CS7320-AI/blob/master/Python%20Code%20Examples/store_agent_state_information.ipynb)

In [7]:
# Your short description of the state and your implementation goes here
# My agent records its location state only when it has found the north-west corner 
# My agent has an array of locations to report where it has previously visited
# The array of locations is only populated after it has found the north-west corner
# Our environment has to be slightly changed to allow for agent objects to be passed instead of functions

In [8]:
class ModelBasedAgent:
    def __init__(self, initial_position = [], name = "Zach's Agent"):
        self.position = initial_position
        self.name = name
        self.cleaned = []
        self.target_direction = "east"
        self.tiles_navigated = 0
    
    def act(self, bumpers, dirty):
        #Two cases where all the bumpers are activated:
        if(bumpers["north"] and bumpers["south"] and bumpers["east"] and bumpers["west"] and dirty): return "suck"
        if(bumpers["north"] and bumpers["south"] and bumpers["east"] and bumpers["west"]): return "noop"
        #In navigation mode, trying to fnd north-west corner
        if(len(self.position) == 0):
            #We have found the north-west corner
            if(bumpers["north"] and bumpers["west"]): 
                self.position = [0,0]
                #first action after finding north-west corner
                if(dirty == True): return "suck"
                if(bumpers["east"] == False): 
                    self.position = [self.position[0],self.position[1]+1]
                    self.tiles_navigated = self.tiles_navigated + 1
                    return "east"
                self.target_direction = "west"
                self.position = [self.position[0]+1,self.position[1]]
                self.tiles_navigated = self.tiles_navigated + 1
                return "south"
            #Haven't found northwest corner. Moving closer to corner
            if(bumpers["north"] == False and bumpers["north"] == False): return random.choice(["north", "west"])
            if(bumpers["north"]): return "west"
            return "north"
        #navigate through room, choosing the actions
        else:
            if(dirty): 
                self.cleaned.append(self.position)
                return "suck"
            if(self.target_direction == "east"):
                if(bumpers["east"] and bumpers["south"]): 
                    self.target_direction = "west"
                    self.position = [self.position[0],self.position[1]-1]
                    self.tiles_navigated = self.tiles_navigated + 1
                    return "west"
                if(bumpers["east"]):
                    self.target_direction = "west"
                    self.position = [self.position[0]+1,self.position[1]]
                    self.tiles_navigated = self.tiles_navigated + 1
                    return "south"
                self.position = [self.position[0],self.position[1]+1]
                self.tiles_navigated = self.tiles_navigated + 1
                return "east"
            if(bumpers["west"] and bumpers["south"]): 
                self.target_direction = "east"
                self.position = [self.position[0],self.position[1]+1]
                self.tiles_navigated = self.tiles_navigated + 1
                return "east"
            if(bumpers["west"]):
                self.target_direction = "east"
                self.position = [self.position[0]+1,self.position[1]]
                self.tiles_navigated = self.tiles_navigated + 1
                return "south"
            self.position = [self.position[0],self.position[1]-1]
            self.tiles_navigated = self.tiles_navigated + 1
            return "west"
    
agent = ModelBasedAgent(name = "Zach's Model Based Agent")
square_environment(agent.act, dims=5, max_steps = 1000)

Shape of room:  (5, 5)
Generated room:
[False False  True False  True]
[False False False  True False]
[False False False False  True]
[False False False False  True]
[False False False False False]
room has 5 dirty tiles
Robot starts at tile: ( 3 , 1 )
step 0 - action: north position: ( 3 , 1 )
step 1 - action: west position: ( 2 , 1 )
step 2 - action: north position: ( 2 , 0 )
step 3 - action: north position: ( 1 , 0 )
step 4 - action: east position: ( 0 , 0 )
step 5 - action: east position: ( 0 , 1 )
step 6 - action: suck position: ( 0 , 2 )
step 7 - action: east position: ( 0 , 2 )
step 8 - action: east position: ( 0 , 3 )
step 9 - action: suck position: ( 0 , 4 )
step 10 - action: south position: ( 0 , 4 )
step 11 - action: west position: ( 1 , 4 )
step 12 - action: suck position: ( 1 , 3 )
step 13 - action: west position: ( 1 , 3 )
step 14 - action: west position: ( 1 , 2 )
step 15 - action: west position: ( 1 , 1 )
step 16 - action: south position: ( 1 , 0 )
step 17 - action: ea

5

## Task 4: Simulation study [3 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results in a suitable format (tables, graphs) and discuss the differences. 

Here is some help with [charts and tables.](https://github.com/mhahsler/CS7320-AI/blob/master/Python%20Code%20Examples/charts_and_tables.ipynb)

In [9]:
# Each agent gets the same environment
def simulated_environment(agent, dims, max_steps, verbose = True):
    num_cleaned = 0
    num_dirty = 0
    energy_used = 0
    results = [0,0,0]
    
    #generating room with each tile having 20% chance of being dirty
    room = array([[random.choice([False, True], p=[0.8, 0.2]) for x in range(dims)] for y in range(dims)])
    if(verbose): print("Shape of room: ", room.shape)
    if(verbose): print("Generated room:")
        
    #count number of dirty tiles
    for x in room: 
        for y in x:
            if(y == True): num_dirty = num_dirty + 1
        if(verbose): 
            print(x)
    if (verbose): print("room has", num_dirty , "dirty tiles")
        
    #randomly assign robot position in environment
    robot_pos = array([random.randint(dims-1), random.randint(dims-1)])
    if(verbose): print("Robot starts at tile: (", robot_pos[0], ",", robot_pos[1], ")")
        
    #initialize robot bumpers
    bumpers = update_bumpers(robot_pos, dims)
    
    #store environment information for resetting environment for other agents
    room_init = room.copy()
    num_dirty_init = num_dirty
    robot_pos_init = robot_pos.copy()
    bumpers_init = bumpers.copy()
    
    #Robot does actions until max_steps or all tiles clean
    #Randomized agent
    for i in range(max_steps):
        if(num_dirty == 0): break
        dirty = room[robot_pos[0],robot_pos[1]]
        action = simple_randomized_agent(bumpers, dirty)
        energy_used = energy_used + 1
        if (verbose): print("step", i , "- action:", action, "position: (", robot_pos[0], ",", robot_pos[1], ")")
        # If robot sucks and tile is dirty, update counters
        if (action == "suck"): 
            if(room[robot_pos[0],robot_pos[1]]):
                room[robot_pos[0],robot_pos[1]] = False
                num_cleaned = num_cleaned + 1
                num_dirty = num_dirty - 1
        # If robot is unable to do anything, break
        elif (action == "noop"):
            if (verbose): print("Robot is unable to do anything. Stopping robot.")
            break
        # Update the position and bumpers for next action
        else:
            robot_pos = update_position(robot_pos, action, bumpers)
            bumpers = update_bumpers(robot_pos, dims)
            
    results[0] = energy_used
    
    #reinitialize for simple reflex agent
    room = room_init.copy()
    num_dirty = num_dirty_init
    robot_pos = robot_pos_init.copy()
    bumpers = bumpers_init.copy()
    num_cleaned = 0
    energy_used = 0
    
    #Simple Reflex Agent
    for i in range(max_steps):
        if(num_dirty == 0): break
        dirty = room[robot_pos[0],robot_pos[1]]
        action = simple_reflex_agent(bumpers, dirty)
        energy_used = energy_used + 1
        if (verbose): print("step", i , "- action:", action, "position: (", robot_pos[0], ",", robot_pos[1], ")")
        # If robot sucks and tile is dirty, update counters
        if (action == "suck"): 
            if(room[robot_pos[0],robot_pos[1]]):
                room[robot_pos[0],robot_pos[1]] = False
                num_cleaned = num_cleaned + 1
                num_dirty = num_dirty - 1
        # If robot is unable to do anything, break
        elif (action == "noop"):
            if (verbose): print("Robot is unable to do anything. Stopping robot.")
            break
        # Update the position and bumpers for next action
        else:
            robot_pos = update_position(robot_pos, action, bumpers)
            bumpers = update_bumpers(robot_pos, dims)
            
    results[1] = energy_used
    
    #reinitialize for Model-based Reflex Agent
    room = room_init.copy()
    num_dirty = num_dirty_init
    robot_pos = robot_pos_init.copy()
    bumpers = bumpers_init.copy()
    num_cleaned = 0
    energy_used = 0
    
    #Model-based Reflex Agent
    for i in range(max_steps):
        if(num_dirty == 0): break
        dirty = room[robot_pos[0],robot_pos[1]]
        action = agent(bumpers, dirty)
        energy_used = energy_used + 1
        if (verbose): print("step", i , "- action:", action, "position: (", robot_pos[0], ",", robot_pos[1], ")")
        # If robot sucks and tile is dirty, update counters
        if (action == "suck"): 
            if(room[robot_pos[0],robot_pos[1]]):
                room[robot_pos[0],robot_pos[1]] = False
                num_cleaned = num_cleaned + 1
                num_dirty = num_dirty - 1
        # If robot is unable to do anything, break
        elif (action == "noop"):
            if (verbose): print("Robot is unable to do anything. Stopping robot.")
            break
        # Update the position and bumpers for next action
        else:
            robot_pos = update_position(robot_pos, action, bumpers)
            bumpers = update_bumpers(robot_pos, dims)
            
    results[2] = energy_used
       
    if (verbose): print("Number of tiles cleaned: ", num_cleaned, ".")
    if (verbose): print("Number of dirty tiles remaining = ", num_dirty, ".")
    if (verbose): print("Amount of energy units consumed: ", energy_used, ".")
    return results

#5x5 square first
simulated_results = []

for i in range(100):
    agent = ModelBasedAgent(name = "Zach's Model Based Agent")
    simulated_results.append(simulated_environment(agent.act, dims=5, max_steps = 1000, verbose = False))
average_results = [0,0,0]
for i in range(100):
    average_results[0] = average_results[0] + simulated_results[i][0]
    average_results[1] = average_results[1] + simulated_results[i][1]
    average_results[2] = average_results[2] + simulated_results[i][2]
average_results[0] = float(average_results[0]/100)
average_results[1] = float(average_results[1]/100)
average_results[2] = float(average_results[2]/100)

print("For a 5x5 square:")
print("Randomized Agent average energy used: ", average_results[0])
print("Simple Reflex Agent average energy used: ", average_results[1])
print("Model-based Reflex Agent average energy used: ", average_results[2])

#10x10 square next
simulated_results = []

for i in range(100):
    agent = ModelBasedAgent(name = "Zach's Model Based Agent")
    simulated_results.append(simulated_environment(agent.act, dims=10, max_steps = 100000, verbose = False))
average_results = [0,0,0]
for i in range(100):
    average_results[0] = average_results[0] + simulated_results[i][0]
    average_results[1] = average_results[1] + simulated_results[i][1]
    average_results[2] = average_results[2] + simulated_results[i][2]
average_results[0] = float(average_results[0]/100)
average_results[1] = float(average_results[1]/100)
average_results[2] = float(average_results[2]/100)

print("For a 10x10 square:")
print("Randomized Agent average energy used: ", average_results[0])
print("Simple Reflex Agent average energy used: ", average_results[1])
print("Model-based Reflex Agent average energy used: ", average_results[2])

#100x100 square last
simulated_results = []

for i in range(100):
    print("Run number: ", i)
    agent = ModelBasedAgent(name = "Zach's Model Based Agent")
    simulated_results.append(simulated_environment(agent.act, dims=100, max_steps = 10000000, verbose = False))
average_results = [0,0,0]
for i in range(100):
    average_results[0] = average_results[0] + simulated_results[i][0]
    average_results[1] = average_results[1] + simulated_results[i][1]
    average_results[2] = average_results[2] + simulated_results[i][2]
average_results[0] = float(average_results[0]/100)
average_results[1] = float(average_results[1]/100)
average_results[2] = float(average_results[2]/100)

print("For a 100x100 square:")
print("Randomized Agent average energy used: ", average_results[0])
print("Simple Reflex Agent average energy used: ", average_results[1])
print("Model-based Reflex Agent average energy used: ", average_results[2])

For a 5x5 square:
Randomized Agent average energy used:  424.02
Simple Refex Agent average energy used:  101.71
Model-based Reflex Agent average energy used:  28.34
For a 10x10 square:
Randomized Agent average energy used:  2988.49
Simple Refex Agent average energy used:  892.88
Model-based Reflex Agent average energy used:  125.5
Run number:  0
Run number:  1
Run number:  2
Run number:  3
Run number:  4
Run number:  5
Run number:  6
Run number:  7
Run number:  8
Run number:  9
Run number:  10
Run number:  11
Run number:  12
Run number:  13
Run number:  14
Run number:  15
Run number:  16
Run number:  17
Run number:  18
Run number:  19
Run number:  20
Run number:  21
Run number:  22
Run number:  23
Run number:  24
Run number:  25
Run number:  26
Run number:  27
Run number:  28
Run number:  29
Run number:  30
Run number:  31
Run number:  32
Run number:  33
Run number:  34
Run number:  35
Run number:  36
Run number:  37
Run number:  38
Run number:  39
Run number:  40
Run number:  41
Run n

Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5      | 424.02           | 101.71              | 28.34                    |
| 10x10    | 2988.49          | 892.88              | 125.5                    |
| 100x100  | 851277.33        | 337302.21           | 12101.64                 |


As was anticipated, the Model-based Reflex Agent performed the best in all three room sizes.
The randomized agent took about 4 times as long as the simple reflex agent and 15 times as long as the model-based reflex agent in the 5x5 room.  
  
In the 10x10 and 100x100 rooms the disparity between the randomized and model-based Reflex Agent grew even larger, at 24 and 70 times, respectively. The 100x100 simulation took much longer to run than the other two as well.



### Cost per Square

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 25       | 16.96            | 4.07                | 1.13                     |
| 100      | 29.88            | 8.93                | 1.13                     |
| 10000    | 85.13            | 33.73               | 1.21                     |

The average cost per square is a good performance metric to show how efficient the robot vaccums are in the environment. The average cost per square for the model-based reflex agent stays around the same for any size of room. We can also see that the cost per square increases for the randomized and simple reflex agents as the room size increases. These observations indicate that the model-based reflex agent's cost is proportional to the number of squares in the room, while the cost for the randomized and simple reflex agents are more complex. For very large rooms, the memory for storing state model-based reflex agent can be worth the cost. If the robots operate on batteries, the randomized robot vacuum would likely need to be charged before finishing cleaning of a larger room.

## Task 5: Robustness of the agent implementations [1 Point] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).  
  
### Rectangular room with unknown size  
  
If the agensts were placed into a rectandular room with unknown size, they would generally take about the same amount of time to complete as a square. All three implementations would clean the room eventually, but the model-based reflex agent would still clean the room the fastest. Since the model-based agent begins cleaning once it reaches the top-right corner of the room, it would still be able to start navigating the room square-by-square until it has navigated the entire room.  
  
### Irregularly-shaped cleaning area
  
If the room has an irregular shape, however, the current implementation of the model-based reflex agent would likely not clean the entire room. Once the model-based reflex agent finds a corner where the north and west bumpers are activated, it may not find the true north-west corner. Also, the model-based reflex agent is only able to travel east, west, or south, so it would never reach any of the northern squares that it would missed. If there were any dirty squares that it missed, the robot would continually travel east and west in the south-most row it found, and would only stop once the max_steps is met.  
  
The randomized and simple reflex agents would both clean the room eventually, although they may take a lot longer than with a rectangle. Since the only way to clean the whole space is for the robot to find and traverse a hallway, the amount of time  that it takes to clean the room would depend on how narrow and long a spaces between rooms is. The closer the space is to the shape of a square, the faster the randomized and simple agents will clean the space.

### Room containing obstacles  
  
If the room is has obstacles, the current implementation of the model-based reflex agent would have problems. Upon reaching an obstacle, the robot would attempt to travel south, if it can, or the opposite direction. It would not clean any squares past the obstacle. Since the robot cannot travel north once it has started cleaning, it will miss any of the dirty tiles that it skipped. If it missed dirty tiles, the agent would continually travel east and west in the south-most row it found, and would only stop once the max_steps is met. If on the way to the north-west corner, the model-based reflex agent encounters an location where the north and west bumpers are activated, it will begin cleaning to the east, potentially missing any dirty squares to the north or west.

The randomized and simple reflex agents would still be able to clean the entire room, and their results would look similar to the performance gathered from the simulation in Task 4. The only time these agents would not be able to clean the whole room is when there is a dirty square that is boxed-in by the walls and object tiles. The more obstacles there are in the room, the longer the randomized and simple reflex agents would take to clean.



## Graduate student advanced task: Obstacles [1 Point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [37]:
#Environment with obstacles

# We need a new bumper updater function
def update_bumpers_obstacles(robot_pos, dims, room):
    bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}
    if(robot_pos[0] == 0): bumpers.update({"north": True})
    if(robot_pos[0] == dims-1): bumpers.update({"south": True})
    if(robot_pos[1] == 0): bumpers.update({"west": True})
    if(robot_pos[1] == dims-1): bumpers.update({"east": True})
    if(robot_pos[0]-1 >= 0 and room[robot_pos[0]-1,robot_pos[1]] == 2): bumpers.update({"north": True})
    if(robot_pos[0]+1 <= dims-1 and room[robot_pos[0]+1,robot_pos[1]] == 2): bumpers.update({"south": True})
    if(robot_pos[1]-1 >= 0 and room[robot_pos[0],robot_pos[1]-1] == 2): bumpers.update({"west": True})
    if(robot_pos[1]+1 <= dims-1 and room[robot_pos[0],robot_pos[1]+1] == 2): bumpers.update({"east": True})
    return bumpers

# Each agent gets the same environment
def obstacle_environment(agent, dims, max_steps, verbose = True):
    num_cleaned = 0
    num_dirty = 0
    num_obstacles = 0
    num_squares = dims * dims
    energy_used = 0
    
    results = [0,0,0]
    
    #generating room with each tile having 20% chance of being dirt and 10% chance of an obstacle
    room = array([[random.choice([0, 1, 2], p=[0.7, 0.2, 0.1]) for x in range(dims)] for y in range(dims)])
    if(verbose): print("Shape of room: ", room.shape)
    if(verbose): print("Generated room:")
        
    #count number of dirty tiles
    for x in room: 
        for y in x:
            if(y == 1): num_dirty = num_dirty + 1
            if(y == 2): num_obstacles = num_obstacles + 1
        if(verbose): 
            print(x)
    if (verbose): print("room has", num_dirty , "dirty tiles and ", num_obstacles, "obstacles")
    
    #We can only place the robot if there is a square with no obstacles.
    if(num_squares == num_obstacles): 
        if (verbose): print("Robot is unable to placed due to entire room being filled with obstacles.")
        return results
        
    #randomly assign robot position in environment
    robot_pos = array([random.randint(dims-1), random.randint(dims-1)])
    while(room[robot_pos[0], robot_pos[1]] == 2):
        robot_pos = array([random.randint(dims-1), random.randint(dims-1)])
    if(verbose): print("Robot starts at tile: (", robot_pos[0], ",", robot_pos[1], ")")
        
    #initialize robot bumpers
    bumpers = update_bumpers_obstacles(robot_pos, dims, room)
    
    #store environment information for resetting environment for other agents
    room_init = room.copy()
    num_dirty_init = num_dirty
    robot_pos_init = robot_pos.copy()
    bumpers_init = bumpers.copy()
    is_clean = 0
    
    #Robot does actions until max_steps or all tiles clean
    #Randomized agent
    for i in range(max_steps):
        if(num_dirty == 0): break
        dirty = room[robot_pos[0],robot_pos[1]]
        action = simple_randomized_agent(bumpers, dirty)
        energy_used = energy_used + 1
        if (verbose): print("step", i , "- action:", action, "position: (", robot_pos[0], ",", robot_pos[1], ")")
        # If robot sucks and tile is dirty, update counters
        if (action == "suck"): 
            if(room[robot_pos[0],robot_pos[1]]):
                room[robot_pos[0],robot_pos[1]] = False
                num_cleaned = num_cleaned + 1
                num_dirty = num_dirty - 1
        # If robot is unable to do anything, break
        elif (action == "noop"):
            if (verbose): print("Robot is unable to do anything. Stopping robot.")
            break
        # Update the position and bumpers for next action
        else:
            robot_pos = update_position(robot_pos, action, bumpers)
            bumpers = update_bumpers_obstacles(robot_pos, dims, room)
            
    if(num_cleaned == num_dirty_init): is_clean = 1
    results[0] = [energy_used, is_clean]
    
    #reinitialize for simple reflex agent
    room = room_init.copy()
    num_dirty = num_dirty_init
    robot_pos = robot_pos_init.copy()
    bumpers = bumpers_init.copy()
    num_cleaned = 0
    energy_used = 0
    is_clean = 0
    
    #Simple Reflex Agent
    for i in range(max_steps):
        if(num_dirty == 0): break
        dirty = room[robot_pos[0],robot_pos[1]]
        action = simple_reflex_agent(bumpers, dirty)
        energy_used = energy_used + 1
        if (verbose): print("step", i , "- action:", action, "position: (", robot_pos[0], ",", robot_pos[1], ")")
        # If robot sucks and tile is dirty, update counters
        if (action == "suck"): 
            if(room[robot_pos[0],robot_pos[1]]):
                room[robot_pos[0],robot_pos[1]] = False
                num_cleaned = num_cleaned + 1
                num_dirty = num_dirty - 1
        # If robot is unable to do anything, break
        elif (action == "noop"):
            if (verbose): print("Robot is unable to do anything. Stopping robot.")
            break
        # Update the position and bumpers for next action
        else:
            robot_pos = update_position(robot_pos, action, bumpers)
            bumpers = update_bumpers_obstacles(robot_pos, dims, room)
            
    if(num_cleaned == num_dirty_init): is_clean = 1
    results[1] = [energy_used, is_clean]
    
    #reinitialize for Model-based Reflex Agent
    room = room_init.copy()
    num_dirty = num_dirty_init
    robot_pos = robot_pos_init.copy()
    bumpers = bumpers_init.copy()
    num_cleaned = 0
    energy_used = 0
    is_clean = 0
    
    #Model-based Reflex Agent
    for i in range(max_steps):
        if(num_dirty == 0): break
        dirty = room[robot_pos[0],robot_pos[1]]
        action = agent(bumpers, dirty)
        energy_used = energy_used + 1
        if (verbose): print("step", i , "- action:", action, "position: (", robot_pos[0], ",", robot_pos[1], ")")
        # If robot sucks and tile is dirty, update counters
        if (action == "suck"): 
            if(room[robot_pos[0],robot_pos[1]]):
                room[robot_pos[0],robot_pos[1]] = False
                num_cleaned = num_cleaned + 1
                num_dirty = num_dirty - 1
        # If robot is unable to do anything, break
        elif (action == "noop"):
            if (verbose): print("Robot is unable to do anything. Stopping robot.")
            break
        # Update the position and bumpers for next action
        else:
            robot_pos = update_position(robot_pos, action, bumpers)
            bumpers = update_bumpers_obstacles(robot_pos, dims, room)
            
    if(num_cleaned == num_dirty_init): is_clean = 1
    results[2] = [energy_used, is_clean]
       
    if (verbose): print("Number of tiles cleaned: ", num_cleaned, ".")
    if (verbose): print("Number of dirty tiles remaining = ", num_dirty, ".")
    if (verbose): print("Amount of energy units consumed: ", energy_used, ".")
    return results

#5x5 agent first
simulated_results = []
total_rooms_cleaned = [0,0,0]

for i in range(100):
    agent = ModelBasedAgent(name = "Zach's Model Based Agent")
    simulated_results.append(obstacle_environment(agent.act, dims=5, max_steps = 2000, verbose = False))
average_results = [0,0,0]
for i in range(100):
    average_results[0] = average_results[0] + simulated_results[i][0][0]
    average_results[1] = average_results[1] + simulated_results[i][1][0]
    average_results[2] = average_results[2] + simulated_results[i][2][0]
    if(simulated_results[i][0][1]): total_rooms_cleaned[0] = total_rooms_cleaned[0] + 1
    if(simulated_results[i][1][1]): total_rooms_cleaned[1] = total_rooms_cleaned[1] + 1
    if(simulated_results[i][2][1]): total_rooms_cleaned[2] = total_rooms_cleaned[2] + 1
average_results[0] = float(average_results[0]/100)
average_results[1] = float(average_results[1]/100)
average_results[2] = float(average_results[2]/100)

print("For a 5x5 square, simulated 100 times:")
print("Randomized Agent average energy used: ", average_results[0], "with ", total_rooms_cleaned[0], " rooms cleaned")
print("Simple Reflex Agent average energy used: ", average_results[1], "with ", total_rooms_cleaned[1], " rooms cleaned")
print("Model-based Reflex Agent average energy used: ", average_results[2], "with ", total_rooms_cleaned[2], " rooms cleaned")

#10x10 agent next
simulated_results = []
total_rooms_cleaned = [0,0,0]

for i in range(100):
    agent = ModelBasedAgent(name = "Zach's Model Based Agent")
    simulated_results.append(obstacle_environment(agent.act, dims=10, max_steps = 15000, verbose = False))
average_results = [0,0,0]
for i in range(100):
    average_results[0] = average_results[0] + simulated_results[i][0][0]
    average_results[1] = average_results[1] + simulated_results[i][1][0]
    average_results[2] = average_results[2] + simulated_results[i][2][0]
    if(simulated_results[i][0][1]): total_rooms_cleaned[0] = total_rooms_cleaned[0] + 1
    if(simulated_results[i][1][1]): total_rooms_cleaned[1] = total_rooms_cleaned[1] + 1
    if(simulated_results[i][2][1]): total_rooms_cleaned[2] = total_rooms_cleaned[2] + 1
average_results[0] = float(average_results[0]/100)
average_results[1] = float(average_results[1]/100)
average_results[2] = float(average_results[2]/100)

print("For a 10x10 square, simulated 100 times:")
print("Randomized Agent average energy used: ", average_results[0], "with ", total_rooms_cleaned[0], " rooms cleaned")
print("Simple Reflex Agent average energy used: ", average_results[1], "with ", total_rooms_cleaned[1], " rooms cleaned")
print("Model-based Reflex Agent average energy used: ", average_results[2], "with ", total_rooms_cleaned[2], " rooms cleaned")

#100x100 agent last
simulated_results = []
total_rooms_cleaned = [0,0,0]

for i in range(10):
    print("Run number: ", i)
    agent = ModelBasedAgent(name = "Zach's Model Based Agent")
    simulated_results.append(obstacle_environment(agent.act, dims=100, max_steps = 3000000, verbose = False))
average_results = [0,0,0]
for i in range(10):
    average_results[0] = average_results[0] + simulated_results[i][0][0]
    average_results[1] = average_results[1] + simulated_results[i][1][0]
    average_results[2] = average_results[2] + simulated_results[i][2][0]
    if(simulated_results[i][0][1]): total_rooms_cleaned[0] = total_rooms_cleaned[0] + 1
    if(simulated_results[i][1][1]): total_rooms_cleaned[1] = total_rooms_cleaned[1] + 1
    if(simulated_results[i][2][1]): total_rooms_cleaned[2] = total_rooms_cleaned[2] + 1
average_results[0] = float(average_results[0]/100)
average_results[1] = float(average_results[1]/100)
average_results[2] = float(average_results[2]/100)

print("For a 100x100 square, simulated 10 times:")
print("Randomized Agent average energy used: ", average_results[0], "with ", total_rooms_cleaned[0], " rooms cleaned")
print("Simple Reflex Agent average energy used: ", average_results[1], "with ", total_rooms_cleaned[1], " rooms cleaned")
print("Model-based Reflex Agent average energy used: ", average_results[2], "with ", total_rooms_cleaned[2], " rooms cleaned")

For a 5x5 square, simulated 100 times:
Randomized Agent average energy used:  515.14 with  95  rooms cleaned
Simple Refex Agent average energy used:  205.0 with  95  rooms cleaned
Model-based Reflex Agent average energy used:  1131.25 with  44  rooms cleaned
For a 10x10 square, simulated 100 times:
Randomized Agent average energy used:  2968.35 with  100  rooms cleaned
Simple Refex Agent average energy used:  1078.82 with  100  rooms cleaned
Model-based Reflex Agent average energy used:  14851.33 with  1  rooms cleaned
Run number:  0
Run number:  1
Run number:  2
Run number:  3
Run number:  4
Run number:  5
Run number:  6
Run number:  7
Run number:  8
Run number:  9
For a 100x100 square, simulated 10 times:
Randomized Agent average energy used:  185444.14 with  6  rooms cleaned
Simple Refex Agent average energy used:  156378.45 with  6  rooms cleaned
Model-based Reflex Agent average energy used:  300000.0 with  0  rooms cleaned


### How the agents react to obstacles in the environment  
  
I added obstacles to our environment by changing the grid to storing an integer instead of boolean values. A value of '0' represents a clean square with no obstacles. A value of '1' represents a dirty square with no obstacles. A value of '2' represents a square with obstacles. In this example, 10% of the total squares have obstacles added, with 20% being dirty. I also added another metric feature that shows how many rooms were cleaned during a simulation, in order to show if the agent finished cleaning or hit the maximum number of steps.  
  
We can see in our results that the randomized and simple reflex agent performed better than the model-based reflex agent. This is because the model-based reflex agent hit the maximum number of steps almost every time. For the 5x5 square, the randomized and simple reflex agents cleaned 95% of the rooms completely, while the model-based reflex agent was only able to clean 44%. For the 10x10 square, the model-based reflex agent was only able to completely clean 1%, while the other agents cleaned 100%. Finally, for the 100x100 square, the model-based reflex agent didn't clean a single room, while the other agents cleaned 60%.
Clearly the model-based reflex agent would need an updated cleaning algorithm.

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5      | 515.14           | 205.0               | 1131.25                  |
| 10x10    | 2968.35          | 1078.82             | 125.5                    |
| 100x100  | 185444.14        | 156378.45           | 300000.0                 |
  
### Improving the agent's handling of obstacles  
  
A fullproof way of cleaning the room (besides squares blocked-in by obstacles and walls) is for the robot to perform a breadth-first search across all of the squares, recording the location of each square visited. The location of the agent would be the position relative to the starting point. The nodes of the search tree would be based on the bumper sensors. If a bumper is set to "False", then a new node is added in the direction of the bumper. Eventually every square would be visited.

The biggest downside to this approach is the amount of memory used. For increasingly large rooms, the amount of space able to be stored might be too large for a robot vacuum. However, for a 100x100 room, this robot should function well. There would certainly be some more efficient algorithms, but this would most likely clean the whole room faster than both of the other agents.
  
## More advanced tasks to think about

You can think about these:

* __Unknown environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square.

* __Utility-based agent:__ Change the environment, so each square has a fixed probability of getting dirty again. We assume the agent has learned this information over time. For the implementation, we give this information to the agent as a 2-dimensional array of probabilities  Cleaning one dirty square produces a utility of 1. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 10000 time steps. This is very tricky!